# settings

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import time
import datetime

import requests
from bs4 import BeautifulSoup

In [ ]:
from google.colab import drive

# Web Scraping module

Set conditions (page, categories, keyword, date & order)

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

In [ ]:
def news_scraper(query, maxpage, start_date, end_date) :

  start = start_date.split("/")
  end = end_date.split("/")
  news_df = pd.DataFrame(columns = ['title', 'datetime', 'article'])
  index = 0

  page = 1

  while page < int(maxpage) : 
    url = "https://find.mk.co.kr/new/search.php?pageNum="+ str(page) + \
          "&cat=&cat1=&media_eco=&pageSize=20&sub=news&dispFlag=OFF&page=news&s_kwd=" + query + \
          "&s_page=news&go_page=&ord=1&ord1=1&ord2=0&s_keyword=" + query + "&s_i_keyword=" + query + \
          "&s_author=&y1=" + start[0] + "&m1=" + start[1] + "&d1=" + start[2] + "&y2=" + end[0] + "&m2=" + end[1] + "&d2=" + end[2] + \
          "&ord=1&area=tt"
    
    page_ = requests.get(url)
    page_ = page_.content
    page_soup = BeautifulSoup(page_, 'html.parser')

    for news_url in page_soup.select(".art_tit > a") :

      try :
        news = requests.get(news_url['href'], headers = headers)
        news.encoding = 'euc-kr'
        news_soup = BeautifulSoup(news.text, 'html.parser')

        title = news_soup.select_one("h1.top_title").text
        try :
          datetime = news_soup.select_one("li.lasttime").text[5:15]
        except :
          datetime = news_soup.select_one("li.lasttime1").text[5:15]

        article = news_soup.select(".art_txt")[0].get_text()
        article.replace('\n', ' ').replace('\t', ' ')
        article = article.replace("function _flash_removeCallback() {}", "")

        news_df.loc[index] = [title, datetime, article]
        index += 1

      except Exception as e :
        continue
      
    page += 1
    print("✔ Page ", page)

  print("👌 Scraping completed!")
    
  return news_df
  

Search for : tesla  
from 2020/01/01 (or before) to 2022/05/01  
Max page : 100

In [ ]:
query = input("Search keyword : ")
maxpage = input("Pages to search : ")
start_date = input("Start date (YYYY/MM/DD) : ")
end_date = input("End date (YYYY/MM/DD) : ")

start_time = time.time()
news_df = news_scraper(query, maxpage, start_date, end_date)
processing_time = time.time() - start_time
times = str(datetime.timedelta(seconds = processing_time))

print("Time spent :", times)

In [ ]:
news_df.shape

(1841, 3)

### Cleaning text

In [ ]:
# Delete image
news_df['article'] = news_df.article.apply(lambda x : x.replace("▶ 여기를 누르시면 크게 보실 수 있습니다", " "))

In [ ]:
# Delete text with [] () 
import re
news_df['article'] = news_df.article.replace(r'\[[^]]*\]', '', regex=True)
news_df['article'] = news_df.article.replace(r'\([^)]*\)', '', regex=True)

In [ ]:
news_df

,title,datetime,article
0,"소비자와 가장 친밀한 브랜드, 디즈니, 테슬라, 애플 순",2022.04.27,"전 세계 브랜드 중 소비자와 가장 친밀한 브랜드는 디즈니, 테슬라, 애플 순으로..."
1,테슬라 '스마트폰 호출'했더니…수십억 비행기에 충돌,2022.04.27,자율 주행하던 테슬라가 40억원대 전용기를 들이받았다.\r26일 영국 미러 등 외...
2,트위터 인수 발표 다음날 테슬라 주가 12% 하락…1년여만 최대,2022.04.27,일론 머스크 테슬라 최고경영자 억만장자 일론 머스크가 이끄는 전기차 업체 테슬라...
3,경기둔화 공포에 짓눌린 뉴욕증시…나스닥 4%↓·테슬라 12%↓(종합),2022.04.27,미국 뉴욕증시가 26일 동시다발적 악재에 따른 경기둔화 공포에 짓눌려 크게 뒷걸음질...
4,"베이조스, 트위터 인수한 머스크에 ""中서 테슬라 복잡해질 것""",2022.04.26,"베이조스, 머스크 트위터 인수에 중국 입김? 세계 최대 전자상거래 업체 아마존 ..."
...,...,...,...
1836,"[Hot-Line] ""LG화학, 테슬라 모델3 공급 계약 가능성…이익기대""",2019.08.26,중국 난징 신장경제개발구에 위치한 LG화학 전기차 배터리 1공장 전경삼성증권은 26...
1837,中 추가관세에 테슬라·포드 '직격탄' 맞는다,2019.08.24,중국이 12월 15일부터 미국산 자동차에 25%의 고율 관세를 부과하게 되면 테슬라...
1838,"""LG화학, 테슬라 中 상하이 전기차공장에 배터리 납품""",2019.08.23,테슬라 중국 상하이 공장 건설 예정 LG화학이 미국 전기자동차 업체 테슬라의 첫 ...
1839,中상하이 자유무역시험구 곱절로 확대…테슬라 '수혜',2019.08.20,상하이 린강에 건설 중인 테슬라 공장중국이 상하이 자유무역시험구 규모를 배 가까이 ...
